In [8]:
import pandas as pd
import numpy as np
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

In [9]:
# Загрузка данных
data = pd.read_csv('news.csv')
data.head()

,text,label
0,Помощник депутата Палаты представителей: мы да...,1
1,"Вы когда-нибудь чувствовали, что ваша жизнь дв...",0
2,Почему правда может привести к увольнению 29 о...,1
3,"Выявлено 15 мирных жителей, погибших в результ...",1
4,Распечатать\nИранская женщина была приговорена...,1


In [10]:
data.shape

(15000, 2)

In [11]:
# Предварительная обработка
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
stop_words = set(stopwords.words('russian'))
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [12]:
def preprocess_text(text):
    text = text.lower()  # Привести к нижнему регистру
    text = "".join([char for char in text if char not in string.punctuation])  # Удалить пунктуацию
    tokens = word_tokenize(text)
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]  # Лемматизация
    return " ".join(tokens)

In [13]:
data['text_processed'] = data['text'].apply(preprocess_text)

In [14]:
# Разделение данных на обучающий и тестовый наборы
X = data['text_processed']
y = data['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [15]:
# Преобразование текста в векторы
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

***ЛОГИСТИЧЕСКАЯ РЕГРЕССИЯ***

In [16]:
# Обучение моделей
lr_classifier = LogisticRegression()
lr_classifier.fit(X_train_tfidf, y_train)

LogisticRegression()

In [17]:
lr_pred = lr_classifier.predict(X_test_tfidf)

In [18]:
lr_accuracy = accuracy_score(y_test, lr_pred)

In [19]:
lr_report = classification_report(y_test, lr_pred)

In [20]:
print("Logistic Regression:", lr_accuracy)

Logistic Regression: 0.9394666666666667


In [21]:
print("Logistic Regression:")
print(lr_report)

Logistic Regression:
              precision    recall  f1-score   support

           0       0.94      0.96      0.95      2090
           1       0.94      0.92      0.93      1660

    accuracy                           0.94      3750
   macro avg       0.94      0.94      0.94      3750
weighted avg       0.94      0.94      0.94      3750



***ДЕРЕВО РЕШЕНИЙ***

In [22]:
dt_classifier = DecisionTreeClassifier()
dt_classifier.fit(X_train_tfidf, y_train)


DecisionTreeClassifier()

In [23]:
dt_pred = dt_classifier.predict(X_test_tfidf)

In [24]:
dt_accuracy = accuracy_score(y_test, dt_pred)

In [25]:
dt_report = classification_report(y_test, dt_pred)

In [26]:
print("Decision Tree Classifier:", dt_accuracy)

Decision Tree Classifier: 0.8616


In [27]:
print("Decision Tree Classifier:")
print(dt_report)

Decision Tree Classifier:
              precision    recall  f1-score   support

           0       0.88      0.88      0.88      2090
           1       0.84      0.84      0.84      1660

    accuracy                           0.86      3750
   macro avg       0.86      0.86      0.86      3750
weighted avg       0.86      0.86      0.86      3750



***ГРАДИЕНТНЫЙ БУСТИНГ***

In [28]:
gb_classifier = GradientBoostingClassifier()
gb_classifier.fit(X_train_tfidf, y_train)

GradientBoostingClassifier()

In [29]:
gb_pred = gb_classifier.predict(X_test_tfidf)

In [30]:
gb_accuracy = accuracy_score(y_test, gb_pred)

In [31]:
gb_report = classification_report(y_test, gb_pred)

In [32]:
print("Gradient Boosting Classifier:", gb_accuracy)

Gradient Boosting Classifier: 0.9117333333333333


In [33]:
print("Gradient Boosting Classifier:")
print(gb_report)

Gradient Boosting Classifier:
              precision    recall  f1-score   support

           0       0.93      0.91      0.92      2090
           1       0.89      0.91      0.90      1660

    accuracy                           0.91      3750
   macro avg       0.91      0.91      0.91      3750
weighted avg       0.91      0.91      0.91      3750



***СЛУЧАЙНЫЙ ЛЕС***

In [34]:
rf_classifier = RandomForestClassifier()
rf_classifier.fit(X_train_tfidf, y_train)

RandomForestClassifier()

In [35]:
rf_pred = rf_classifier.predict(X_test_tfidf)


In [36]:
rf_accuracy = accuracy_score(y_test, rf_pred)

In [37]:
rf_report = classification_report(y_test, rf_pred)

In [38]:
print("Random Forest Classifier:", rf_accuracy)

Random Forest Classifier: 0.8946666666666667


In [39]:
print("Random Forest Classifier:")
print(rf_report)

Random Forest Classifier:
              precision    recall  f1-score   support

           0       0.86      0.96      0.91      2090
           1       0.95      0.81      0.87      1660

    accuracy                           0.89      3750
   macro avg       0.91      0.89      0.89      3750
weighted avg       0.90      0.89      0.89      3750



In [40]:
# Функция для тестирования моделей
def test_model(input_text):
    # Векторизация входного текста
    input_vector = tfidf_vectorizer.transform([input_text])

    # Предсказания для каждой модели
    lr_pred = lr_classifier.predict(input_vector)
    dt_pred = dt_classifier.predict(input_vector)
    gb_pred = gb_classifier.predict(input_vector)
    rf_pred = rf_classifier.predict(input_vector)

    return {"Logistic Regression": "True" if lr_pred[0] else "False",
            "Decision Tree": "True" if dt_pred[0] else "False",
            "Gradient Boosting": "True" if gb_pred[0] else "False",
            "Random Forest": "True" if rf_pred[0] else "False"}

In [56]:
news = str(input())
test_model(news)

ЛОНДОН. Головокружительное празднование «дня независимости» сменилось политическими и экономическими потрясениями, и в отношении бывшего мэра Лондона Бориса Джонсона и других лидеров успешной кампании по голосованию за выход Великобритании из Европейского Союза стало особенно ясно одно: они не было плана на то, что будет дальше. В те дни, когда Великобритания проголосовала за выход из блока, лидеры движения часто выглядели так, как будто они не ожидали победы и не были готовы справиться с последствиями. Столкнувшись с размахом решения, они были заняты отказом от обещаний, данных во время кампании, и снижением ожиданий. Они не смогли выступить единым фронтом или ответить на основные вопросы. Их неустойчивая деятельность усугубила ощущение политического хаоса в Британии и, возможно, беспорядков на финансовых рынках. И это подорвало их доверие и авторитет, поскольку г-н Джонсон готовит свою заявку на пост премьер-министра и ведет Великобританию к новым отношениям с континентом. Ставки выс

{'Logistic Regression': 'False',
 'Decision Tree': 'True',
 'Gradient Boosting': 'True',
 'Random Forest': 'False'}